In [1]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key,
                    consumer_secret,
                    access_token,
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [2]:
#BBC, CBS, CNN, Fox News, New York Times
user_id = ["@BBC", "@CBS", "@CNN", "@FoxNews", "@nytimes"]

results_list = []

for user in user_id:
    counter = 1
    
    for item in tweepy.Cursor(api.user_timeline, id = user).items(100):
        
        tweet = json.dumps(item._json, indent = 3)
        tweet = json.loads(tweet)
        text = tweet["text"]
        
        # Run Vader Analysis on each tweet
        compound = analyzer.polarity_scores(text)["compound"]
        positive = analyzer.polarity_scores(text)["pos"]
        neutral = analyzer.polarity_scores(text)["neu"]
        negative = analyzer.polarity_scores(text)["neg"]
            
        user_results = {"Username": user, "Date Created": tweet["created_at"], 
                    "Tweet Count": counter, "Compound Score": compound, "Postive Score": positive, 
                    "Neutral Score": neutral,"Negative Score": negative, "Text": text}
    
        results_list.append(user_results)
    
        counter += 1


In [3]:
# Create DataFrame from Results List

results_df = pd.DataFrame.from_dict(results_list)
results_df.head()

,Compound Score,Date Created,Negative Score,Neutral Score,Postive Score,Text,Tweet Count,Username
0,0.4215,Sat Sep 29 20:32:03 +0000 2018,0.000,0.743,0.257,Chile has launched a huge hiking route through...,1,@BBC
1,0.4215,Sat Sep 29 17:02:12 +0000 2018,0.000,0.865,0.135,🐣😍 Conservationists have been monitoring the p...,2,@BBC
2,-0.7003,Sat Sep 29 16:01:07 +0000 2018,0.293,0.707,0.000,'My children bought back the car I sold to pay...,3,@BBC
3,0.5499,Sat Sep 29 15:54:38 +0000 2018,0.000,0.871,0.129,RT @BBCEngland: Benny the beluga whale is spen...,4,@BBC
4,0.7088,Sat Sep 29 15:02:04 +0000 2018,0.000,0.671,0.329,Always start by playing paper! 👊✋✌️\n\nThis is...,5,@BBC


In [ ]:
plt.scatter("Tweet Count", "Compound Score", marker="o", facecolors="green", edgecolors="black", alpha=0.80)
#need to figure out how to tell to color for each
plt.title("Sentiment Analysis of Tweets")
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")

plt.grid(True)
plt.ylim(0,130)
plt.xlim(0,100)

#plt.savefig("Twitter Scatterplot.png")
plt.show()

In [ ]:
# Notes

# Target User Accounts
target_user = ("@DalaiLama", "@realdonaldtrump", "@katyperry")

# List for dictionaries of results
results_list = []

# Loop through each user
for user in target_user:

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 pages of tweets (total 200 tweets)
    for x in range(1, 11):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user, page=x)

        # Loop through all tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)

    # Create a dictionaty of results
    user_results = {
        "Username": user,
        "Compound Score": np.mean(compound_list),
        "Postive Score": np.mean(positive_list),
        "Neutral Score": np.mean(neutral_list),
        "Negative Score": np.mean(negative_list)
    }

    # Append dictionary to list
    results_list.append(user_results)

    # Print the Averages
    print(f"User: {user}")
    print(f"Compound: {np.mean(compound_list):.3f}")
    print(f"Positive: {np.mean(positive_list):.3f}")
    print(f"Neutral: {np.mean(neutral_list):.3f}")
    print(f"Negative: {np.mean(negative_list):.3f}")